<a href="https://colab.research.google.com/github/Balenm/Bike_Rentals_PySpark/blob/main/High_Performance_Computing_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Spark

In [ ]:
# This block of code is useful to install and configure latest version of PySpark on Jupyter Notebook (Stand-alone)
!rm -f spark-3.3.[01]-bin-hadoop3.tgz* 
!rm -rf spark-3.3.[01]-bin-hadoop3

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar -xf spark-3.4.0-bin-hadoop3.tgz

#----------------------- Initiating the Session ----------------------------
!pip3 install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
# the next line gives us 'local' mode. try 'local[2]' to use 2 cores or 'master:NNNN' to run on Spark standalone cluster at port NNNN
spark_conf = SparkConf().setMaster('local[*]').setAppName('Cycle_hire')
sc = SparkContext(conf=spark_conf)
# see what we have by examining the Spark User Interface
from pyspark.sql import *
from pyspark.sql.functions import *
SparkSession.builder.getOrCreate()

In [1]:
%%capture
!pip install pyspark

Importing Apache spark libraries

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

Loading the dataset and unzip the files:

In [3]:
%%capture
!wget https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip
!unzip cyclehireusagestats-2014.zip

Checking the directory ans showing the files:

In [4]:
!ls

'10a. Journey Data Extract 14Sep14-27Sep14.csv'
'10b. Journey Data Extract 28Sep14-11Oct14.csv'
'11a. Journey Data Extract 12Oct14-08Nov14.csv'
'11b. Journey Data Extract 12Oct14-08Nov14.csv'
'12a. Journey Data Extract 09Nov14-06Dec14.csv'
'12b. Journey Data Extract 09Nov14-06Dec14.csv'
'13a. Journey Data Extract 07Dec14-21Dec14.csv'
'13b. Journey Data Extract 22Dec14-03Jan15.csv'
'14. Journey Data Extract 08Dec13-04Jan14.csv'
'1. Journey Data Extract 05Jan14-02Feb14.csv'
'2. Journey Data Extract 03Feb14-01Mar14.csv'
'3. Journey Data Extract 02Mar14-31Mar14.csv'
'4. Journey Data Extract 01Apr14-26Apr14.csv'
'5. Journey Data Extract 27Apr14-24May14.csv'
'6. Journey Data Extract 25May14-21Jun14.csv'
'7. Journey Data Extract 22Jun14-19Jul14.csv'
'8a Journey Data Extract 20Jul14-31Jul14.csv'
'8b Journey Data Extract 01Aug14-16Aug14.csv'
'9a Journey Data Extract 17Aug14-31Aug14.csv'
'9b Journey Data Extract 01Sep14-13Sep14.csv'
 cyclehireusagestats-2014.zip
 sample_data




1.   Autumn starts from 1st october to 30th November.
2.   Spring starts from 20th March untill 31 May.



In [5]:
file="./*Journey*csv"
spark = SparkSession.builder.appName("bikes").getOrCreate()
cycling_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load(file))

In [6]:
cycling_df.printSchema()

root
 |-- Rental Id: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Bike Id: string (nullable = true)
 |-- End Date: string (nullable = true)
 |-- EndStation Id: string (nullable = true)
 |-- EndStation Name: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- StartStation Id: string (nullable = true)
 |-- StartStation Name: string (nullable = true)



In [7]:
cycling_df= cycling_df.withColumn("Start_Date", to_date(split(cycling_df["Start Date"], " ")[0], "dd/MM/yyyy"))
cycling_df= cycling_df.withColumn("Start_Time", split(cycling_df["Start Date"], " ")[1])

In [8]:
cycling_df.show(1)

+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+----------+
|Rental Id|Duration|Bike Id|        End Date|EndStation Id|     EndStation Name|      Start Date|StartStation Id|   StartStation Name|Start_Date|Start_Time|
+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+----------+
| 34263367|    1080|   9076|24/06/2014 00:57|          695|Islington Green, ...|24/06/2014 00:39|            311|Foley Street, Fit...|2014-06-24|     00:39|
+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+----------+
only showing top 1 row



In [9]:
cycling_df.printSchema()

root
 |-- Rental Id: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Bike Id: string (nullable = true)
 |-- End Date: string (nullable = true)
 |-- EndStation Id: string (nullable = true)
 |-- EndStation Name: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- StartStation Id: string (nullable = true)
 |-- StartStation Name: string (nullable = true)
 |-- Start_Date: date (nullable = true)
 |-- Start_Time: string (nullable = true)



In [10]:
cycling_aut_spr_df = cycling_df.filter(cycling_df["Start_Date"].between("2014-03-20", "2014-05-30") |
                                       cycling_df["Start_Date"].between("2014-09-01", "2014-11-30")).alias('cycling_aut_spr_df')

In [11]:
spring_df = cycling_aut_spr_df.filter(month("Start_Date").isin([3,4,5]))
autumn_df = cycling_aut_spr_df.filter(month("Start_Date").isin([9,10,11]))

In [12]:
autumn_df.show(2)

+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+----------+
|Rental Id|Duration|Bike Id|        End Date|EndStation Id|     EndStation Name|      Start Date|StartStation Id|   StartStation Name|Start_Date|Start_Time|
+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+----------+
| 37875851|     120|   3229|28/09/2014 00:02|          450|Jubilee Street, S...|28/09/2014 00:00|            443|Philpot Street, W...|2014-09-28|     00:00|
| 37875861|    1080|   3283|28/09/2014 00:18|          645|Great Suffolk Str...|28/09/2014 00:00|             14|Belgrove Street ,...|2014-09-28|     00:00|
+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+----------+
only showing top 2 rows



In [13]:
spring_duration = spring_df.select("Duration").toPandas().to_numpy().ravel()
autumn_duration = autumn_df.select("Duration").toPandas().to_numpy().ravel()

In [29]:
from scipy.stats import ttest_ind
ttest_duration, p_value_duration = ttest_ind(a=spring_duration.astype(np.int64), b=autumn_duration.astype(np.int64), equal_var = False)

In [30]:
if p_value_duration <0.05: 
  print ("We reject the null hypothesis: there is a significant difference in the mean duration of bike rides between the autumn and spring seasons")
else:
  print ("We accept the null hypothesis: there is no significant difference in the mean duration of bike rides between the autumn and spring seasons")

We reject the null hypothesis: there is a significant difference in the mean duration of bike rides between the autumn and spring seasons


# Second Hypothesis
Importing the second dataset

In [17]:
%%capture
!wget https://raw.githubusercontent.com/Balenm/Bike_Rentals_PySpark/main/london_weather.csv

In [18]:
weather_df = spark.read.csv("/content/london_weather.csv",header=True, inferSchema=True)
weather_df = weather_df.withColumnRenamed("Date", "Start_Date")
weather_df= weather_df.withColumn("Start_Date", to_date(weather_df["Start_Date"], "yyyyMMdd"))

In [19]:
weather_df.show(2)

+----------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
|Start_Date|cloud_cover|sunshine|global_radiation|max_temp|mean_temp|min_temp|precipitation|pressure|snow_depth|
+----------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
|1979-01-01|        2.0|     7.0|            52.0|     2.3|     -4.1|    -7.5|          0.4|101900.0|       9.0|
|1979-01-02|        6.0|     1.7|            27.0|     1.6|     -2.6|    -7.5|          0.0|102530.0|       8.0|
+----------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
only showing top 2 rows



In [20]:
spring_df =  spring_df.join(weather_df, on="Start_Date")
autumn_df =  autumn_df.join(weather_df, on="Start_Date")

In [21]:
spring_df.show(3)

+----------+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
|Start_Date|Rental Id|Duration|Bike Id|        End Date|EndStation Id|     EndStation Name|      Start Date|StartStation Id|   StartStation Name|Start_Time|cloud_cover|sunshine|global_radiation|max_temp|mean_temp|min_temp|precipitation|pressure|snow_depth|
+----------+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
|2014-05-27| 33253206|    1140|  11439|27/05/2014 00:59|          695|Islington Green, ...|27/05/2014 00:40|             81|Great Titchfield ...|     00:40|        7.0|     0.0|            81.0|    16.1|     12.0|    10.6|       

In [22]:
autumn_df.show(3)

+----------+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
|Start_Date|Rental Id|Duration|Bike Id|        End Date|EndStation Id|     EndStation Name|      Start Date|StartStation Id|   StartStation Name|Start_Time|cloud_cover|sunshine|global_radiation|max_temp|mean_temp|min_temp|precipitation|pressure|snow_depth|
+----------+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+----------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
|2014-09-28| 37875851|     120|   3229|28/09/2014 00:02|          450|Jubilee Street, S...|28/09/2014 00:00|            443|Philpot Street, W...|     00:00|        2.0|     6.2|           127.0|    19.2|     19.7|    14.7|       

In [25]:
spring_df.select("mean_temp")

+---------+
|mean_temp|
+---------+
|     12.0|
|     14.6|
|     14.6|
+---------+
only showing top 3 rows



In [26]:
spring_weather = spring_df.select("mean_temp").toPandas().to_numpy().ravel()
autumn_weather = autumn_df.select("mean_temp").toPandas().to_numpy().ravel()

In [32]:
ttest_weather, p_value_weather = ttest_ind(a=spring_weather.astype(np.int64), b=autumn_weather.astype(np.int64), equal_var = False)

In [34]:
if p_value_weather <0.05: 
  print ("We reject the null hypothesis: there is a significant difference in the mean weather between the autumn and spring seasons")
else:
  print ("We accept the null hypothesis: there is no a significant difference in the mean weather between the autumn and spring seasons")

We reject the null hypothesis: there is a significant difference in the mean weather between the autumn and spring seasons
